In [2]:
from sympy import *
from latex2sympy2 import latex2sympy
init_printing()

# Be careful with the lagrangian and the length. 
# They're both L's but not the same

m, L, g, n, i, k, t, t_0, tp, tpp = symbols("m L g n i k t t_0 tp tpp")
# m for mass, L for the length of the string, g for gravity, 
# n, i and k are indices


# n = int(input("Enter the number of bobs: "))
n = 2
t = IndexedBase('t')
tp = IndexedBase('tp')


Lg_tex = r'\frac{1}{2} m L^2 \sum_{i=1}^n\left(\left(\sum_{k=1}^i \dot{\theta_k} \cos \left(\theta_k\right)\right)^2+\left(\sum_{k=1}^i \dot{\theta_k} \sin \left(\theta_k\right)\right)^2\right)+m g L\left(\sum_{i=1}^n \cos \left(\theta_i\right)\right)'
Lg = latex2sympy(Lg_tex)
Lg_sympy = (1/2)*m*L**2*Sum(Sum(tp[k]*sin(t[k]), (k, 1, i)).doit()**2 + Sum(tp[k]*cos(t[k]), (k, 1, i)).doit()**2, (i, 1, n)).doit() + m*g*L*Sum(cos(t[i]), (i, 1, n)).doit()

# The Euler-Lagrange equation has two parts, 
# I'm gonna consider the LHS as the shorter side and the RHS as the longer side



Lagrangian_LHS = []
Lagrangian_RHS = []
solutions = [] 
# The elements of this array are the equations for each of the accelerations
# Then we need to solve the system of equations in the solutions array to get each in their own terms


for i in range(n):
    LHS = str(diff(Lg_sympy, t[i + 1]))
    RHS = str(diff(Lg_sympy, tp[i + 1]))

    for j in range(n):
        old_tp = f'tp[{j+1}]'
        new_tp = f'tp{j+1}(t_0)'
        RHS = RHS.replace(old_tp, new_tp)
        LHS = LHS.replace(old_tp, new_tp)
        old_t = f't[{j+1}]'
        new_t = f't{j+1}(t_0)'
        RHS = RHS.replace(old_t, new_t)
        LHS = LHS.replace(old_t, new_t)
        Function(f'tp{j+1}')(t_0)
        Function(f't{j+1}')(t_0)
        
    LHS = sympify(LHS)
    Lagrangian_LHS.append(LHS)
    RHS = diff(RHS, t_0).doit()
    Lagrangian_RHS.append(RHS)




In [3]:
for i in range(n):
    euler_lagrange = Eq(Lagrangian_LHS[i] - Lagrangian_RHS[i], 0)
    solution = solve(euler_lagrange, f"Derivative(tp{i+1}(t_0), t_0)")
    solutions.append(solution)

In [4]:
equations = []
accelerations = []

for i in range(n):
    accelerations.append(sympify(f"Derivative(tp{i+1}(t_0), t_0)"))

for i in range(n):
    equations.append(Eq(solutions[i][0], accelerations[i]))

a = accelerations

In [5]:
solutions_acceleration = solve(equations, (a[0], a[1]))
acceleration_equations = []
for i in range(n):
    acceleration_equations.append(solutions_acceleration[sympify("Derivative(tp1(t_0), t_0)")])

# Code for replacing the convoluted variables like tp1(t_0) to something a bit more readable like tp1

for i in range(n):
    old_tp = f'tp{i+1}(t_0)'
    new_tp = f'tp{i+1}'
    old_t = f't{i+1}(t_0)'
    new_t = f't{i+1}'
    old_ddt = f"Derivative(t{i+1}, t_0)"
    new_ddt = f"tp{i+1}"
    t_weird = f"t[{i+1}]"
    t_cool = f"t{i+1}"

    for j in range(n):
        acceleration_equations[j] = acceleration_equations[j].replace(sympify(old_tp), sympify(new_tp))
        acceleration_equations[j] = acceleration_equations[j].replace(sympify(old_t), sympify(new_t))
        acceleration_equations[j] = acceleration_equations[j].replace(sympify(old_ddt), sympify(new_ddt))


variables = []
for i in range(n):
    variables.append(sympify(f"t{i+1}"))
for i in range(n):
    variables.append(sympify(f"tp{i+1}"))
variables.append(L)
variables.append(g)
variables.append(m)
for i in range(n):
    acceleration_equations[i] = lambdify(variables, acceleration_equations[i])

In [6]:
# Generating the dataset
from random import uniform

solutions = acceleration_equations
N = 2000 # Number of epochs
inputs = []
outputs = []

for i in range(N):
    def AllBobs():
    # This is the code that calculates the angles for ALL bobs, not just the nth one
    #
    # data_iteration_inputs = []
    # data_iteration_outputs = []
    # for j in range(N):
    #     initial_angle = uniform(-180, 180)
    #     initial_velocity = 0
    #     time = uniform(0, 5)
    #     initial_acceleration = solutions[j](initial_angle, initial_velocity)

    #     angle_final = 0.5 * initial_acceleration * time ** 2
    #     data_iteration_inputs.append([initial_angle, initial_velocity, time])
    #     data_iteration_outputs.append(angle_final)
        print("filler text")
    
    # you have to set these variables up yourself!!

    t1 = uniform(-180, 180)
    t2 = uniform(-180, 180)
    tp1 = 0
    tp2 = 0
    time = uniform(0, 5)
    g = uniform(0, 20)
    L = uniform(1, 10)
    m = uniform(1, 100)
    

    tpp1 = solutions[0](t1, t2, tp1, tp2, L, g, m)
    tpp2 = solutions[1](t1, t2, tp1, tp2, L, g, m)

    t1_final = 0.5 * tpp1 * time ** 2
    t2_final = 0.5 * tpp2 * time ** 2

    inputs.append([t1, t2, g, L, m, time])
    outputs.append(t2_final)